# Iteration 2: Post stroke unit destination stage 2

## Plain English Summary

In the model length of stay is determined not just by patient type, but also patient destination post ASU.  For example Early Supported Discharge greatly reduces length of stay.  As such the model is designed to sample destination as the patient arrives.  



## LLM Prompt

::::{admonition} LLM Prompt
:::{include} ../04_prompts_stage1/inputs/02_iteration.md
:::
::::

## Date and time of input to LLM

> 1045 28th June

## Observations on the LLM Output

* The variable `post_asu_probabilities` has been added into the class `PatientType` in the constructor.

* A method has been added to the `PatientType` class with a choice function which works as a discrete-empirical.

* The percentages were converted into probabilities in class `ASUModel` in the PatientType instances. The proportions for each patient type are correct.

```python
    def sample_post_asu_destination(self):
        return self.rng.choice(['Rehab', 'ESD', 'Other'], p=self.post_asu_probabilities)
```

## Sources reported by the LLM

No sources reported

## LLM Output

In [3]:
import simpy
import numpy as np

class PatientType:
    def __init__(self, name, interarrival_time, post_asu_probabilities):
        self.name = name
        self.interarrival_time = interarrival_time
        self.count = 0
        self.rng = np.random.default_rng()
        self.post_asu_probabilities = post_asu_probabilities

    def generate_interarrival_time(self):
        return self.rng.exponential(self.interarrival_time)

    def sample_post_asu_destination(self):
        return self.rng.choice(['Rehab', 'ESD', 'Other'], p=self.post_asu_probabilities)

class ASUModel:
    def __init__(self, env, patient_params=None, run_length=5 * 365):
        self.env = env
        self.run_length = run_length
        self.total_arrivals = 0
        
        if patient_params is None:
            self.patient_types = {
                'Stroke': PatientType('Stroke', 1.2, [0.24, 0.13, 0.63]),
                'TIA': PatientType('TIA', 9.3, [0.01, 0.01, 0.98]),
                'Complex Neurological': PatientType('Complex Neurological', 3.6, [0.11, 0.05, 0.84]),
                'Other': PatientType('Other', 3.2, [0.05, 0.10, 0.85])
            }
        else:
            self.patient_types = {
                name: PatientType(name, params['interarrival_time'], params['post_asu_probabilities'])
                for name, params in patient_params.items()
            }

    def run(self):
        for patient_type in self.patient_types.values():
            self.env.process(self.patient_generator(patient_type))
        self.env.run(until=self.run_length)

    def patient_generator(self, patient_type):
        while True:
            interarrival_time = patient_type.generate_interarrival_time()
            yield self.env.timeout(interarrival_time)
            
            self.total_arrivals += 1
            patient_type.count += 1
            patient_id = self.total_arrivals - 1

            post_asu_destination = patient_type.sample_post_asu_destination()

            print(f"Time {self.env.now:.2f}: Patient {patient_id} ({patient_type.name}) arrived")
            print(f"  Total arrivals: {self.total_arrivals}")
            print(f"  {patient_type.name} arrivals: {patient_type.count}")
            print(f"  Post-ASU destination: {post_asu_destination}")
            print(f"  Next {patient_type.name} arrival in {interarrival_time:.2f} days")
            print()

            # Patient immediately leaves the model
            # No additional processing needed

def main(patient_params=None, run_length=5*365):
    env = simpy.Environment()
    model = ASUModel(env, patient_params, run_length)
    model.run()

    print("\nSimulation completed.")
    print(f"Total simulation time: {model.env.now:.2f} days")
    print(f"Total patient arrivals: {model.total_arrivals}")
    for patient_type in model.patient_types.values():
        print(f"Total {patient_type.name} arrivals: {patient_type.count}")

if __name__ == "__main__":
    main()


Time 0.52: Patient 0 (Complex Neurological) arrived
  Total arrivals: 1
  Complex Neurological arrivals: 1
  Post-ASU destination: Other
  Next Complex Neurological arrival in 0.52 days

Time 1.27: Patient 1 (Other) arrived
  Total arrivals: 2
  Other arrivals: 1
  Post-ASU destination: Other
  Next Other arrival in 1.27 days

Time 1.97: Patient 2 (Complex Neurological) arrived
  Total arrivals: 3
  Complex Neurological arrivals: 2
  Post-ASU destination: Other
  Next Complex Neurological arrival in 1.44 days

Time 2.57: Patient 3 (Complex Neurological) arrived
  Total arrivals: 4
  Complex Neurological arrivals: 3
  Post-ASU destination: Rehab
  Next Complex Neurological arrival in 0.60 days

Time 2.93: Patient 4 (Other) arrived
  Total arrivals: 5
  Other arrivals: 2
  Post-ASU destination: ESD
  Next Other arrival in 1.66 days

Time 3.46: Patient 5 (Stroke) arrived
  Total arrivals: 6
  Stroke arrivals: 1
  Post-ASU destination: Other
  Next Stroke arrival in 3.46 days

Time 4.20: P

## Testing

* **Extreme value test 1**: TIA, Compelex Neuro, Other, have their inter-arrival time is set to $M$ a very large number
    * Expected result: The only type of patient to arrive to the model is "Stroke"
    * Actual result (PASS): The only type of patient to arrive to the model is "Accident and Emergency."
* **Extreme value test 2**: All patient types have have their inter-arrival time is set to $M$ a very large number
    * Expected result: No patients arrive to the model.
    * Actual result (PASS): No patients arrive to the model.
* **Different run length**: The run length of the model is set to 10 days.
    * Expected result: The model should runs no longer than 240 simulated time periods
    * Actual result (PASS): The model runs no longer than 240 simulated time periods

### Extreme value test 1

The model uses hard coded parameters.  The most simple way to modify the code for this test was to change hard coded parameters for $M$

**Result: PASS**

In [19]:
M = 10_000_000

In [23]:
def test_1(large_number):
    # Create the simulation environment
    if __name__ == "__main__":
        asu_arrival_rates = {
            'Stroke': {'interarrival_time': 1.2, 'post_asu_probabilities': None},
            'TIA': {'interarrival_time': large_number, 'post_asu_probabilities': None},
            'Complex Neurological': {'interarrival_time': large_number, 'post_asu_probabilities': None},
            'Other': {'interarrival_time': large_number, 'post_asu_probabilities': None}
        }
        main(asu_arrival_rates, run_length=2*365)  # Run for 2 years with custom parameters


In [24]:
test_1(M)

Time 1.73: Patient 0 (Stroke) arrived
  Total arrivals: 1
  Stroke arrivals: 1
  Post-ASU destination: Rehab
  Next Stroke arrival in 1.73 days

Time 3.69: Patient 1 (Stroke) arrived
  Total arrivals: 2
  Stroke arrivals: 2
  Post-ASU destination: ESD
  Next Stroke arrival in 1.96 days

Time 6.85: Patient 2 (Stroke) arrived
  Total arrivals: 3
  Stroke arrivals: 3
  Post-ASU destination: Rehab
  Next Stroke arrival in 3.16 days

Time 8.72: Patient 3 (Stroke) arrived
  Total arrivals: 4
  Stroke arrivals: 4
  Post-ASU destination: Other
  Next Stroke arrival in 1.86 days

Time 8.79: Patient 4 (Stroke) arrived
  Total arrivals: 5
  Stroke arrivals: 5
  Post-ASU destination: ESD
  Next Stroke arrival in 0.07 days

Time 9.74: Patient 5 (Stroke) arrived
  Total arrivals: 6
  Stroke arrivals: 6
  Post-ASU destination: Rehab
  Next Stroke arrival in 0.95 days

Time 10.50: Patient 6 (Stroke) arrived
  Total arrivals: 7
  Stroke arrivals: 7
  Post-ASU destination: Rehab
  Next Stroke arrival in

### Extreme value test 2

* The code is again modified so that all patient type also have a mean inter-arrival rate of $M$

**Result: PASS**

In [27]:
def test_2(large_number):
    # Create the simulation environment
    if __name__ == "__main__":
        asu_arrival_rates = {
            'Stroke': {'interarrival_time': large_number, 'post_asu_probabilities': None},
            'TIA': {'interarrival_time': large_number, 'post_asu_probabilities': None},
            'Complex Neurological': {'interarrival_time': large_number, 'post_asu_probabilities': None},
            'Other': {'interarrival_time': large_number, 'post_asu_probabilities': None}
        }
        main(asu_arrival_rates, run_length=2*365)  # Run for 2 years with custom parameters


In [28]:
test_2(M)


Simulation completed.
Total simulation time: 730.00 days
Total patient arrivals: 0
Total Stroke arrivals: 0
Total TIA arrivals: 0
Total Complex Neurological arrivals: 0
Total Other arrivals: 0


### Different run length test

The model has again been modified to use the original code and parameters generated by the LLM.

In [31]:
TEN_DAYS = 10

In [32]:
def test_3(run_length):
    # Create the simulation environment
    if __name__ == "__main__":
        main(run_length=run_length)  # Run for 2 years with custom parameters


In [33]:
test_3(TEN_DAYS)

Time 0.57: Patient 0 (Stroke) arrived
  Total arrivals: 1
  Stroke arrivals: 1
  Post-ASU destination: Other
  Next Stroke arrival in 0.57 days

Time 1.57: Patient 1 (Complex Neurological) arrived
  Total arrivals: 2
  Complex Neurological arrivals: 1
  Post-ASU destination: Rehab
  Next Complex Neurological arrival in 1.57 days

Time 2.05: Patient 2 (TIA) arrived
  Total arrivals: 3
  TIA arrivals: 1
  Post-ASU destination: Other
  Next TIA arrival in 2.05 days

Time 2.26: Patient 3 (Complex Neurological) arrived
  Total arrivals: 4
  Complex Neurological arrivals: 2
  Post-ASU destination: Other
  Next Complex Neurological arrival in 0.69 days

Time 3.00: Patient 4 (Other) arrived
  Total arrivals: 5
  Other arrivals: 1
  Post-ASU destination: Other
  Next Other arrival in 3.00 days

Time 3.10: Patient 5 (Stroke) arrived
  Total arrivals: 6
  Stroke arrivals: 2
  Post-ASU destination: Other
  Next Stroke arrival in 2.53 days

Time 3.99: Patient 6 (Other) arrived
  Total arrivals: 7
 